In [1]:
%cd /home/aoq559/dev/transformer/eap/edge-attribution-patching

from IPython import get_ipython
ipython = get_ipython()
if ipython is not None:
    ipython.magic("%load_ext autoreload")
    ipython.magic("%autoreload 2")

import torch

import torch as t
from torch import Tensor
import einops

from transformer_lens import HookedTransformer

from eapp.eap_wrapper import EAP

from jaxtyping import Float

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/home/aoq559/dev/transformer/eap/edge-attribution-patching


/tmp/ipykernel_49049/2219647472.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%load_ext autoreload")
/tmp/ipykernel_49049/2219647472.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("%autoreload 2")


Device: cuda


# Model Setup

In [2]:
model = HookedTransformer.from_pretrained(
    'EleutherAI/pythia-12b-deduped-v0',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
    n_devices=7,
    move_to_device=True,
    dtype='float16'
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/aoq559/miniconda3/envs/eap/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-12b-deduped-v0 into HookedTransformer


In [3]:
# load data
import yaml
import pickle
import os
class DotDict(dict):
    """ Dot notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
yaml_file_path = "./conf/config.yaml"
with open(yaml_file_path, "r") as f:
    args = DotDict(yaml.safe_load(f))

file_name = args.data_dir
file_name += '/' + str(args.model)
file_name += '/intervention_' + str(args.n_shots) + '_shots_max_' + str(args.max_n) + '_' + args.representation
file_name += '_further_templates' if args.extended_templates else ''
file_name += '.pkl'
print(file_name)

with open(file_name, 'rb') as f:
    intervention_list = pickle.load(f)
print("Loaded data from", file_name)
if args.debug_run:
    intervention_list = intervention_list[:2]

/shared-network/shared/2024_ml_master/data/EleutherAI/pythia-12b-deduped-v0/intervention_1_shots_max_20_words_further_templates.pkl
Loaded data from /shared-network/shared/2024_ml_master/data/EleutherAI/pythia-12b-deduped-v0/intervention_1_shots_max_20_words_further_templates.pkl


In [6]:
clean_dataset.__dict__

{'seed': 1,
 'has_been_flipped': False,
 'prompt_type': 'mixed',
 'templates': ['When [A] and [B] got a [OBJECT] at the [PLACE], [B] decided to give it to [A]',
  'Then, [A] and [B] were thinking about going to the [PLACE]. [B] wanted to give a [OBJECT] to [A]',
  'Then, [B] and [A] went to the [PLACE]. [B] gave a [OBJECT] to [A]',
  'After [A] and [B] went to the [PLACE], [B] gave a [OBJECT] to [A]',
  'When [B] and [A] got a [OBJECT] at the [PLACE], [B] decided to give it to [A]',
  'After [B] and [A] went to the [PLACE], [B] gave a [OBJECT] to [A]',
  'Then, [B] and [A] were thinking about going to the [PLACE]. [B] wanted to give a [OBJECT] to [A]',
  'Then, [A] and [B] had a lot of fun at the [PLACE]. [B] gave a [OBJECT] to [A]',
  'Then, [A] and [B] went to the [PLACE]. [B] gave a [OBJECT] to [A]',
  'Then, [A] and [B] had a long argument, and afterwards [B] said to [A]',
  'Then, [B] and [A] had a long argument, and afterwards [B] said to [A]',
  'Then, [B] and [A] had a lot of f

# Dataset Setup

In [4]:
from ioi_dataset import IOIDataset, format_prompt, make_table
N = 25
clean_dataset = IOIDataset(
    prompt_type='mixed',
    N=N,
    tokenizer=model.tokenizer,
    prepend_bos=False,
    seed=1,
    device=device
)
corr_dataset = clean_dataset.gen_flipped_prompts('ABC->XYZ, BAB->XYZ')

make_table(
  colnames = ["IOI prompt", "IOI subj", "IOI indirect obj", "ABC prompt"],
  cols = [
    map(format_prompt, clean_dataset.sentences),
    model.to_string(clean_dataset.s_tokenIDs).split(),
    model.to_string(clean_dataset.io_tokenIDs).split(),
    map(format_prompt, clean_dataset.sentences),
  ],
  title = "Sentences from IOI vs ABC distribution",
)

                                      Sentences from IOI vs ABC distribution                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ IOI prompt                              ┃ IOI subj ┃ IOI indirect obj ┃ ABC prompt                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ When Victoria and Jane got a snack at   │ Jane     │ Victoria         │ When Victoria and Jane got a snack at   │
│ the store, Jane decided to give it to   │          │                  │ the store, Jane decided to give it to   │
│ Victoria                                │          │                  │ Victoria                                │
│                                         │          │                  │                                         │
│ When Sullivan and Rose got a necklace   │ Sullivan │ Rose             │ When Sullivan and Rose got a necklace   │
│ at the garden, Sullivan decided to give │          │                  │ at the garden, Sullivan decided to give │
│ it to Rose                              │          │                  │ it to Rose                              │
│                                         │          │                  │                                         │
│ When Alan and Alex got a drink at the   │ Alex     │ Alan             │ When Alan and Alex got a drink at the   │
│ store, Alex decided to give it to Alan  │          │                  │ store, Alex decided to give it to Alan  │
│                                         │          │                  │                                         │
│ Then, Jessica and Crystal had a long    │ Jessica  │ Crystal          │ Then, Jessica and Crystal had a long    │
│ argument, and afterwards Jessica said   │          │                  │ argument, and afterwards Jessica said   │
│ to Crystal                              │          │                  │ to Crystal                              │
│                                         │          │                  │                                         │
│ Then, Jonathan and Kevin were working   │ Kevin    │ Jonathan         │ Then, Jonathan and Kevin were working   │
│ at the school. Kevin decided to give a  │          │                  │ at the school. Kevin decided to give a  │
│ necklace to Jonathan                    │          │                  │ necklace to Jonathan                    │
│                                         │          │                  │                                         │
└─────────────────────────────────────────┴──────────┴──────────────────┴─────────────────────────────────────────┘

# Metric Setup

In [4]:
intervention = intervention_list[0]
print(intervention.res_base_tok)
intervention.__dict__

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


[42380]


{'op3_pos': 12,
 'operator_word': None,
 'operands_alt': 'eight six five',
 'operands_base': 'eight six five',
 'operator_pos': None,
 'op2_pos': 10,
 'op1_pos': 8,
 'res_alt_tok': [42380],
 'res_base_tok': [42380],
 'res_string': None,
 'res_base_string': 'nineteen',
 'res_alt_string': 'nineteen',
 'device': 'cpu',
 'multitoken': False,
 'is_llama': False,
 'is_opt': False,
 'representation': 'words',
 'extended_templates': True,
 'template_id': '-',
 'n_vars': 2,
 'base_string': 'eight + six + five =',
 'alt_string': 'eight + six + five =',
 'few_shots': 'five + four + two = eleven. ',
 'few_shots_t2': ' ',
 'equation': '({x}+{y}+{z})',
 'enc': GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-12b-deduped-v0', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  ad

In [18]:
intervention = intervention_list[0]
with torch.no_grad():
    clean_logits = model(intervention.base_string_tok).cpu().numpy()
    corrupt_logits = model(intervention.alt_string_tok).cpu().numpy()
    clean_logit = clean_logits[:, -1, intervention.res_base_tok[0]]
    corrupt_logit = corrupt_logits[:, -1, intervention.pred_res_alt_tok[0]]
    logit_diff = corrupt_logit - clean_logit
    print(clean_logit)
    print(corrupt_logit)

    
    # clean_logit_argmax = torch.argmax(clean_logits, dim=2)
    # corrupt_logit_argmax = torch.argmax(corrupt_logits, dim=2)
    # print(clean_logit_argmax)
    # next_word_index = clean_logit_argmax[0][-1]
    # next_word = model.tokenizer.convert_ids_to_tokens(next_word_index.item())
    # print(next_word)
    
    # clean_prediction = torch.argmax(clean_logits, dim=2)
    # print(clean_prediction)
    # next_word_index = clean_prediction[0][-1]
    # next_word = tokenizer.convert_ids_to_tokens(next_word_index.item())
    # print(next_word_index)
    # print(next_word)
    # print(torch.argmax(clean_logits[:, -1, :]))
    # print(corrupt_logits.shape)

[18.06]
[12.125]


In [27]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-12b-deduped-v0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
tokenizer.convert_ids_to_tokens(27291)

'Ġthirteen'

In [7]:
def ave_logit_diff(
    logits: Float[Tensor, 'batch seq d_vocab'],
    ioi_dataset: IOIDataset,
    per_prompt: bool = False
):
    '''
        Return average logit difference between correct and incorrect answers
    '''
    # Get logits for indirect objects
    batch_size = logits.size(0)
    io_logits = logits[range(batch_size), ioi_dataset.word_idx['end'][:batch_size], ioi_dataset.io_tokenIDs[:batch_size]]
    s_logits = logits[range(batch_size), ioi_dataset.word_idx['end'][:batch_size], ioi_dataset.s_tokenIDs[:batch_size]]
    # Get logits for subject
    logit_diff = io_logits - s_logits
    return logit_diff if per_prompt else logit_diff.mean()

with t.no_grad():
    clean_logits = model(clean_dataset.toks)
    corrupt_logits = model(corr_dataset.toks)
    clean_logit_diff = ave_logit_diff(clean_logits, clean_dataset).item()
    corrupt_logit_diff = ave_logit_diff(corrupt_logits, corr_dataset).item()

def ioi_metric(
    logits: Float[Tensor, "batch seq_len d_vocab"],
    corrupted_logit_diff: float = corrupt_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    ioi_dataset: IOIDataset = clean_dataset
 ):
    patched_logit_diff = ave_logit_diff(logits, ioi_dataset)
    return (patched_logit_diff - corrupted_logit_diff) / (clean_logit_diff - corrupted_logit_diff)

def negative_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -ioi_metric(logits)
    
# Get clean and corrupt logit differences
with t.no_grad():
    clean_metric = ioi_metric(clean_logits, corrupt_logit_diff, clean_logit_diff, clean_dataset)
    corrupt_metric = ioi_metric(corrupt_logits, corrupt_logit_diff, clean_logit_diff, corr_dataset)

print(f'Clean direction: {clean_logit_diff}, Corrupt direction: {corrupt_logit_diff}')
print(f'Clean metric: {clean_metric}, Corrupt metric: {corrupt_metric}')

Clean direction: 2.805162191390991, Corrupt direction: 1.4692586660385132
Clean metric: 1.0, Corrupt metric: 0.0


# Run Experiment

In [10]:
model.reset_hooks()

graph = EAP(
    model,
    clean_dataset.toks,
    corr_dataset.toks,
    ioi_metric,
    upstream_nodes=["mlp", "head"],
    downstream_nodes=["mlp", "head"],
    batch_size=25
)

top_edges = graph.top_edges(n=10, abs_scores=True)
for from_edge, to_edge, score in top_edges:
    print(f'{from_edge} -> [{round(score, 3)}] -> {to_edge}')

Saving activations requires 0.0004 GB of memory per token


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:03<00:00,  3.42s/it]


head.9.9 -> [-0.029] -> head.11.10.q
head.10.7 -> [0.029] -> head.11.10.q
head.5.5 -> [0.021] -> head.8.6.v
head.9.9 -> [-0.019] -> head.10.7.q
head.5.5 -> [-0.019] -> mlp.5
mlp.0 -> [0.018] -> head.6.9.q
mlp.0 -> [-0.015] -> mlp.4
head.5.5 -> [-0.015] -> head.6.9.q
mlp.0 -> [-0.014] -> head.11.10.k
head.3.0 -> [-0.014] -> mlp.5


In [13]:
top_edges

[('head.9.9', 'head.11.10.q', -0.0291709266602993),
 ('head.10.7', 'head.11.10.q', 0.02903885766863823),
 ('head.5.5', 'head.8.6.v', 0.020961817353963852),
 ('head.9.9', 'head.10.7.q', -0.01940404810011387),
 ('head.5.5', 'mlp.5', -0.018816370517015457),
 ('mlp.0', 'head.6.9.q', 0.01845172606408596),
 ('mlp.0', 'mlp.4', -0.0152858542278409),
 ('head.5.5', 'head.6.9.q', -0.014597196131944656),
 ('mlp.0', 'head.11.10.k', -0.013949546962976456),
 ('head.3.0', 'mlp.5', -0.013543691486120224)]